# JSON examples

In [48]:
import json
import math

In [55]:
data = [{'a':22, 'b':345, 'c':'comment'},
        {'a':5.1e8, 'b':0, 'c':'another comment'}]
filename = 'test.json'
with open(filename, "w") as f:
    json.dump(data, f)


In [56]:
with open(filename, "r") as f:
    content = json.load(f)
print(content)

[{'a': 22, 'b': 345, 'c': 'comment'}, {'a': 510000000.0, 'b': 0, 'c': 'another comment'}]


In [57]:
# Test missing values and null values (None)
data2 ={'a':math.nan, 'b':None}

In [58]:
print(json.dumps(data2, indent=2, allow_nan=True))

{
  "a": NaN,
  "b": null
}


In [59]:
print(json.dumps(data2, indent=2, allow_nan=False))

ValueError: Out of range float values are not JSON compliant: nan

In [60]:
# Build a larger dataset and write custom encoder to handle NaN values
data.append(data2)
print(data)

def nan2Value(obj):
    if isinstance(obj, dict):
        return {k:nan2Value(v) for k,v in obj.items()}
    elif isinstance(obj, list):
        return [nan2Value(v) for v in obj]
    elif isinstance(obj, float) and math.isnan(obj):
        return -9999.
    return obj

class CustomEncoder(json.JSONEncoder):
    # Note: documentation says we can overwrite the default method. However,
    # this is called only for data types not known to encode, and NaN is known.
    def encode(self, obj, *args, **kwargs):
        return super().encode(nan2Value(obj), *args, **kwargs)
        
print(json.dumps(data, indent=2, allow_nan=False, cls=CustomEncoder))


[{'a': 22, 'b': 345, 'c': 'comment'}, {'a': 510000000.0, 'b': 0, 'c': 'another comment'}, {'a': nan, 'b': None}]
[
  {
    "a": 22,
    "b": 345,
    "c": "comment"
  },
  {
    "a": 510000000.0,
    "b": 0,
    "c": "another comment"
  },
  {
    "a": -9999.0,
    "b": null
  }
]
